# This notebook will be mainly used for the Captsone Project. This is the first part of this project.

In [1]:
import pandas as pd
import numpy as np
import requests
import json
from pandas.io.json import json_normalize 

In [2]:
print('Hello Captsone Project Course!')

Hello Captsone Project Course!


# Segmenting and Clustering Neighborhoods in Toronto

###### Importing the necessary libraries for this project.

In [3]:
from bs4 import BeautifulSoup
import urllib.request
import csv
import sys

###### Simple scraper function using BeautifulSoup

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
selector = '.wikitable'

def scrape(url, selector):
    i=0
    
    with urllib.request.urlopen(url) as f:
        response = f.read().decode('utf-8')
    soup = BeautifulSoup(response, 'html.parser')
    table = soup.select(selector)
    for row in table[0].find_all('tr'):
        cell_list = []
        for cell in row.find_all(['th', 'td']):
            cell_list.append(cell.text)
        cell_list[2] = cell_list[2][:-1]
        if len(cell_list):
            
            if (i == 0):
                df = pd.DataFrame(columns=cell_list)
                i+=1
            elif(cell_list[1] == 'Not assigned'):
                continue
            elif(cell_list[2] == 'Not assigned'):
                cell_list[2] = cell_list[1]
                df.loc[i-1] = cell_list
                i+=1
            else:
                df.loc[i-1] = cell_list
                i+=1
            
    return df


df = scrape(url, selector)



###### Giving the final shape to the DataFrame

In [5]:
df['Neighbourhood'] = df[['Postcode', 'Borough', 'Neighbourhood']].groupby(['Postcode', 'Borough'])['Neighbourhood'].transform(lambda x: ','.join(x))
df = df.drop_duplicates()
df = df.rename(columns={'Postcode':'Postal Code', 'Neighbourhood':'Neighborhood'})

In [6]:
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park


In [7]:
df.shape

(103, 3)

###### Adding Latitude and Longitude to the DataFrame

In [9]:
lldf = pd.read_csv('Geospatial_Coordinates.csv')

In [10]:
lldf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
finaldf = pd.merge(df, lldf, on="Postal Code")

In [12]:
finaldf.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


In [13]:
from geopy.geocoders import Nominatim

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

## Analyzing & Visualizing Toronto Data

#### Creating map of Toronto

In [14]:
map_toronto = folium.Map(location=[43.654260, -79.360636], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(finaldf['Latitude'], finaldf['Longitude'], finaldf['Borough'], finaldf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

##### Lets look at Downtown Toronto Data

In [15]:
downtown_df = finaldf[finaldf['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
2,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


#### Creating Map of Downtown Toronto

In [16]:
map_downtown = folium.Map(location=[43.654260, -79.360636], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_df['Latitude'], downtown_df['Longitude'], downtown_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

#### Defining Foursquare Credentials and Version

In [17]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


#### Let's explore the first neighborhood in our dataframe

In [18]:
downtown_df.loc[0, 'Neighborhood']

neighborhood_latitude = downtown_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = downtown_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = downtown_df.loc[0, 'Neighborhood'] # neighborhood name

#### Creating a function that gets top 100 venues that are in a neighborhood within a radius of 500 meters

In [24]:
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Calling the function above

In [25]:
downtown_venues = getNearbyVenues(names=downtown_df['Neighborhood'],
                                   latitudes=downtown_df['Latitude'],
                                   longitudes=downtown_df['Longitude']
                                  )

Harbourfront
Queen's Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city
Church and Wellesley


In [26]:
downtown_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [27]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",12,12,12,12,12,12
"Cabbagetown,St. James Town",44,44,44,44,44,44
Central Bay Street,83,83,83,83,83,83
"Chinatown,Grange Park,Kensington Market",91,91,91,91,91,91
Christie,17,17,17,17,17,17
Church and Wellesley,81,81,81,81,81,81
"Commerce Court,Victoria Hotel",100,100,100,100,100,100


## Analyze each neighborhood

In [28]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Group neighborhoods by taking the mean of the frequency of occurrence of each category

In [29]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,...,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.01
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.00
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.083333,0.083333,0.166667,0.166667,0.166667,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,Central Bay Street,0.012048,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012048,0.000000,...,0.000000,0.00,0.00,0.00,0.012048,0.000000,0.000000,0.012048,0.000000,0.00
5,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.054945,0.000000,0.054945,0.010989,0.000000,0.00
6,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,Church and Wellesley,0.012346,0.012346,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,0.000000,...,0.012346,0.00,0.00,0.00,0.000000,0.000000,0.012346,0.000000,0.012346,0.00
8,"Commerce Court,Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,...,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.00
9,"Design Exchange,Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,...,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.00


#### Print each neighborhood along with the top 5 most common venues

In [30]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
              venue  freq
0       Coffee Shop  0.07
1               Bar  0.04
2        Steakhouse  0.04
3              Café  0.04
4  Asian Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.04
3          Steakhouse  0.04
4      Farmers Market  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0    Airport Lounge  0.17
1   Airport Service  0.17
2  Airport Terminal  0.17
3     Boat or Ferry  0.08
4           Airport  0.08


----Cabbagetown,St. James Town----
                venue  freq
0          Restaurant  0.07
1         Coffee Shop  0.07
2  Italian Restaurant  0.05
3                Park  0.05
4              Bakery  0.05


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1  Italian Restaurant  0.05
2                Café  0.05
3        B

#### Putting the above in a pandas dataframe

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### New dataframe with top 10 venues for each neighborhood

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Bar,Steakhouse,Café,Thai Restaurant,Hotel,Sushi Restaurant,Asian Restaurant,Burger Joint,Clothing Store
1,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Cheese Shop,Beer Bar,Seafood Restaurant,Farmers Market,Bakery,Café,Gourmet Shop
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Terminal,Airport Service,Boutique,Sculpture Garden,Harbor / Marina,Airport Food Court,Airport,Boat or Ferry,Falafel Restaurant
3,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Pub,Italian Restaurant,Bakery,Pizza Place,Café,Park,Caribbean Restaurant,Breakfast Spot
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Japanese Restaurant,Sandwich Place,Burger Joint,Juice Bar,Ice Cream Shop,Salad Place,Chinese Restaurant


## Cluster Neighborhoods

#### k-means clustering into 5 clusters

In [33]:
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 0, 3, 0, 0, 0], dtype=int32)

#### New dataframe  that includes the cluster

In [34]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Pub,Park,Bakery,Restaurant,Mexican Restaurant,Breakfast Spot,Café,Beer Store,Performing Arts Venue
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,4,Coffee Shop,Park,Gym,Café,Italian Restaurant,Sandwich Place,Juice Bar,Burger Joint,Burrito Place,Diner
2,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Diner,Bakery,Plaza
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Restaurant,Cocktail Bar,Hotel,Diner,Bakery,Beer Bar,Cosmetics Shop,Breakfast Spot
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Steakhouse,Cheese Shop,Beer Bar,Seafood Restaurant,Farmers Market,Bakery,Café,Gourmet Shop


#### Creating map of clusters

In [36]:
# create map
map_clusters = folium.Map(location=[43.654260, -79.360636], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1

In [37]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Pub,Park,Bakery,Restaurant,Mexican Restaurant,Breakfast Spot,Café,Beer Store,Performing Arts Venue
2,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Diner,Bakery,Plaza
3,Downtown Toronto,0,Café,Coffee Shop,Restaurant,Cocktail Bar,Hotel,Diner,Bakery,Beer Bar,Cosmetics Shop,Breakfast Spot
4,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Steakhouse,Cheese Shop,Beer Bar,Seafood Restaurant,Farmers Market,Bakery,Café,Gourmet Shop
5,Downtown Toronto,0,Coffee Shop,Café,Italian Restaurant,Japanese Restaurant,Sandwich Place,Burger Joint,Juice Bar,Ice Cream Shop,Salad Place,Chinese Restaurant
7,Downtown Toronto,0,Coffee Shop,Bar,Steakhouse,Café,Thai Restaurant,Hotel,Sushi Restaurant,Asian Restaurant,Burger Joint,Clothing Store
8,Downtown Toronto,0,Coffee Shop,Aquarium,Café,Hotel,Italian Restaurant,Scenic Lookout,Brewery,Restaurant,Fried Chicken Joint,Sporting Goods Shop
9,Downtown Toronto,0,Coffee Shop,Hotel,Café,Bar,Seafood Restaurant,Steakhouse,Lounge,Italian Restaurant,Restaurant,Gastropub
10,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Seafood Restaurant,Steakhouse,Italian Restaurant,Gym,Gastropub,Deli / Bodega
11,Downtown Toronto,0,Café,Sandwich Place,Italian Restaurant,Bar,Bakery,Restaurant,Japanese Restaurant,Bookstore,Beer Store,Dessert Shop


#### Cluster 2

In [38]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,1,Park,Playground,Trail,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


#### Cluster 3

In [39]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,2,Airport Lounge,Airport Terminal,Airport Service,Boutique,Sculpture Garden,Harbor / Marina,Airport Food Court,Airport,Boat or Ferry,Falafel Restaurant


#### Cluster 4

In [40]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,3,Grocery Store,Café,Park,Athletics & Sports,Gas Station,Italian Restaurant,Diner,Nightclub,Candy Store,Restaurant


#### Cluster 5

In [41]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,4,Coffee Shop,Park,Gym,Café,Italian Restaurant,Sandwich Place,Juice Bar,Burger Joint,Burrito Place,Diner
